In [2]:
import pandas as pd 
import numpy as np 
import os

In [3]:
df = pd.read_csv('RaceResultsAll.csv', index_col=0)
df

/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/272211675.py:1: DtypeWarning: Columns (2,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('RaceResultsAll.csv', index_col=0)


,Date,Place,BFSP Rank,Ind SP,Ind SP Decimal,Time,Runners,Track Name,Class (GB only),Distance,...,Unnamed: 1993,Unnamed: 1994,Unnamed: 1995,Unnamed: 1996,Unnamed: 1997,Unnamed: 1998,Unnamed: 1999,Unnamed: 2000,Unnamed: 2001,Unnamed: 2002
0,2022-03-01,1,1.0,9/4F,3.25,13:00:00,12.0,Catterick,5.0,2m3½f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-01,2,8.0,25/1,26.00,13:00:00,12.0,Catterick,5.0,2m3½f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-03-01,3,3.0,7/1,8.00,13:00:00,12.0,Catterick,5.0,2m3½f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-01,4,9.0,28/1,29.00,13:00:00,12.0,Catterick,5.0,2m3½f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-03-01,5,5.0,7/1,8.00,13:00:00,12.0,Catterick,5.0,2m3½f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,2022-12-26,4,6.0,12/1,13.00,17:20:00,8.0,Wolverhampton(AW),5.0,1m4f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
832,2022-12-26,5,5.0,7/1,8.00,17:20:00,8.0,Wolverhampton(AW),5.0,1m4f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
833,2022-12-26,6,8.0,40/1,41.00,17:20:00,8.0,Wolverhampton(AW),5.0,1m4f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
834,2022-12-26,7,7.0,33/1,34.00,17:20:00,8.0,Wolverhampton(AW),5.0,1m4f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_columns = list(df.columns)
unamed_columns = []
for column in df_columns:
    if 'Unnamed' in column:
        unamed_columns.append(column)

i = 1
renaming_dict = {}
for column in unamed_columns:
    renaming_dict[column] = 'RaceTime'+str(i)
    i += 1

df = df.rename(columns=renaming_dict)

# What features do I want to look at and collect for each horse node
Historical for the Horse:
- Win Rate (%)
- Place Rate (%); percentage of time it places top 3
- Average Placing (place / number of horses racing)
- Average winning distance to horse infront
- Average winning distnce to winning horse
- Wins in the last 5 races
- Average place in the last 5 races
- Place Rate for last 5 races
- Number of wins total
- Number of races raced

Do the above for:
- The Jockey
- The Trainer
- The Horse for that distance
- The Horse for similar distance
- The Jockey for that distance
- The Jockey for similar distance
- The Trainer for that distance
- The Trainer for similar distance
- The Horse on that going
- The Jockey on that going
- The Traner on that going
- The Jockey with that Horse
- The Jockey with that Trainer


Horse Attribute:
- Pace
- Weight
- Age
- Official Rating
- Time since last Race

Jockey Attribute:
- Time since last Race

Betting:
- Betfair SP
- Industry SP

# What features do I want to look at and collect for the edges - i.e. comparing horses

In [8]:
from datetime import datetime
import pandas as pd

indx_with_better_data = 23550

df = pd.read_csv('RaceResultsOrdered.csv', index_col=0)
df['DateTime'] = df[['Date', 'Time']].apply(lambda x: datetime.strptime(x['Date'] + " " + x['Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
df = df.sort_values(by="DateTime")
df = df.loc[df['Runners'] > 4]
df = df.dropna(subset=['Place'])
df = df.reset_index()

/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/3823019886.py:6: DtypeWarning: Columns (2,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('RaceResultsOrdered.csv', index_col=0)


In [9]:
# Helper features
non_numeric_places = {
    'F', 
    'PU',
    '0',
    'SU', 
    'REF', 
    'UR',
    'DSQ',
    'RO',
    'BD',
    'RR',
    'nan',
    'CO',
    'VOI'
}

frac_to_float = {
    "½": 0.5,
}

def distance_to_metres(distance: str):
    mile_to_m = 1609.34
    furlong_to_m = 201.168

    if 'm' in distance and 'f' in distance:
        miles = distance.split('m')[0]
        furlong = distance.split('m')[1].split('f')[0]
        try:
            miles_num = float(miles)
        except:
            if len(miles) > 1:
                miles_num = float(miles[0]) + frac_to_float[miles[1]]
            else:
                miles_num = frac_to_float[miles[0]]
        try:
            furlong_num = float(furlong)
        except:
            if len(furlong) > 1:
                furlong_num = float(furlong[0]) + frac_to_float[furlong[1]]
            else:
                furlong_num = frac_to_float[furlong[0]]
    elif 'm' in distance:
        miles = distance.split('m')[0]
        try:
            miles_num = float(miles)
        except:
            if len(miles) > 1:
                miles_num = float(miles[0]) + frac_to_float[miles[1]]
            else:
                miles_num = frac_to_float[miles[0]]

        furlong_num = 0
    else:
        furlong = distance.split('f')[0]
        try:
            furlong_num = float(furlong)
        except:
            if len(furlong) > 1:
                furlong_num = float(furlong[0]) + frac_to_float[furlong[1]]
            else:
                furlong_num = frac_to_float[furlong[0]]

        miles_num = 0

    return (miles_num*mile_to_m) + (furlong_num*furlong_to_m)

def place_to_int(x: pd.Series):
    if type(x['Place']) == int:
        return x['Place']
    elif type(x['Place']) == str and x['Place'] not in non_numeric_places:
        return int(x['Place'])
    else:
        return int(x['Runners'])
    
df['Runners'] = df.groupby(by=["DateTime", "Track Name"])['Place'].transform(lambda x: len(x))

df['PlaceInt'] = df[['Place', 'Runners']].apply(lambda x: place_to_int(x), axis=1)
df['win'] = df['PlaceInt'] == 1
df['PlaceTop3'] = df['PlaceInt'] <= 3
df['PlaceNorm'] = df['PlaceInt'] / df['Runners']
df['one'] = 1
df['DistanceMetres'] = df['Distance'].apply(lambda x: distance_to_metres(x))

num_bins = 5
df['distance_bin'] = pd.cut(df['DistanceMetres'], bins=num_bins)

In [10]:
column_names_data = {
    'Pace', 
    'Age', 
    'WeightKg', 
    'OffR', 
    'TimeSinceLastRaceForHorse', 
    'TimeSinceLastRaceForJockey',
    'betfair_implied_prob',
    'betfair_implied_prob_norm',
    'betfair_implied_prob_norm_inv',
    'industry_implied_prob',
    'industry_implied_prob_norm',
    'industry_implied_prob_norm_inv'
}

# Get historical features for a certain group
def get_historical_for_group(df: pd.DataFrame, by: list[str]):

    df.sort_values(by=by)
    name_append = "".join(by)

    # Get win result of last race
    df["win_last_by_"+name_append] = (
        df.groupby(by)["win"]
        .transform(lambda x: x.shift(1)).fillna(0)
    )
    column_names_data.add("win_last_by_"+name_append)

    # Get Place norm of last race
    df["place_last_by_"+name_append] = (
        df.groupby(by)["PlaceNorm"]
        .transform(lambda x: x.shift(1))
    )
    column_names_data.add("place_last_by_"+name_append)

    # Get Number of wins in the last 5
    df["win_last_5_by_"+name_append] = (
        df.groupby(by)["win"]
        .transform(lambda x: x.shift(1).rolling(5, min_periods=1).sum()).fillna(0)
    )
    column_names_data.add("win_last_5_by_"+name_append)

    # Get average placing in the last 5 races
    df["avg_place_last_5_by_"+name_append] = (
        df.groupby(by)["PlaceNorm"]
        .transform(lambda x: x.shift(1).rolling(5, min_periods=1).mean())
    )
    column_names_data.add("avg_place_last_5_by_"+name_append)

    # Get Number of wins in the last 3
    df["win_last_3_by_"+name_append] = (
        df.groupby(by)["win"]
        .transform(lambda x: x.shift(1).rolling(3, min_periods=1).sum()).fillna(0)
    )
    column_names_data.add("win_last_3_by_"+name_append)

    # Get average placing in the last 3 races
    df["avg_place_last_3_by_"+name_append] = (
        df.groupby(by)["PlaceNorm"]
        .transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
    )
    column_names_data.add("avg_place_last_3_by_"+name_append)

    # Get the win number up to that race
    df['win_number_by_'+name_append] = (
        df.groupby(by)["win"]
        .transform(lambda x: x.shift(1).cumsum()).fillna(0)
    )
    column_names_data.add("win_number_by_"+name_append)

    # Get average placing so far
    df["place_cum"] = df.groupby(by)["PlaceNorm"].transform(lambda x: x.shift(1).cumsum())
    df["one_cum"] = df.groupby(by)["one"].transform(lambda x: x.shift(1).cumsum())

    df["avg_place_by_"+name_append] = df["place_cum"] / df["one_cum"]
    df["avg_place_by_"+name_append] = df["avg_place_by_"+name_append]
    df = df.drop(columns=["place_cum"]) 
    column_names_data.add("avg_place_by_"+name_append)

    # Get win rate up to that race
    df["win_cum"] = df.groupby(by)["win"].transform(lambda x: x.shift(1).cumsum())

    df["win_rate_by_"+name_append] = df["win_cum"] / df["one_cum"]
    df["win_rate_by_"+name_append] = df["win_rate_by_"+name_append]
    df = df.drop(columns=["win_cum"])
    column_names_data.add("win_rate_by_"+name_append)

    # Get number of races up to race
    df['num_races_by_'+name_append] = df.groupby(by)["one"].transform(lambda x: x.shift(1).cumsum())
    column_names_data.add("num_races_by_"+name_append)

    # Get number of times placed in top 3
    df['win_number_by_'+name_append] = (
        df.groupby(by)["PlaceTop3"]
        .transform(lambda x: x.shift(1).cumsum()).fillna(0)
    )
    column_names_data.add("win_number_by_"+name_append)

    # Get place top 3 rate up to that race
    df["PlaceTop3_cum"] = df.groupby(by)["PlaceTop3"].transform(lambda x: x.shift(1).cumsum())

    df["placetop3_rate_by_"+name_append] = df["PlaceTop3_cum"] / df["one_cum"]
    df["placetop3_rate_by_"+name_append] = df["placetop3_rate_by_"+name_append].fillna(0)
    df = df.drop(columns=["PlaceTop3_cum", "one_cum"])
    column_names_data.add("placetop3_rate_by_"+name_append)

    # Get number of times placed in top 3 in the last 5 races
    df['placeTop3_number_last_5_by_'+name_append] = (
        df.groupby(by)["PlaceTop3"]
        .transform(lambda x: x.shift(1).rolling(5, min_periods=1).sum()).fillna(0)
    )
    column_names_data.add("placeTop3_number_last_5_by_"+name_append)

    # Get number of times placed in top 3 in the last 3 races
    df['placeTop3_number_last_3_by_'+name_append] = (
        df.groupby(by)["PlaceTop3"]
        .transform(lambda x: x.shift(1).rolling(3, min_periods=1).sum()).fillna(0)
    )
    column_names_data.add("placeTop3_number_last_3_by_"+name_append)

    # Get win result of last race
    df["placeTop3_number_last_by_"+name_append] = (
        df.groupby(by)["PlaceTop3"]
        .transform(lambda x: x.shift(1)).fillna(0)
    )
    column_names_data.add("placeTop3_number_last_by_"+name_append)

    return df

In [14]:
df['PlaceInt']

0          1
1         12
2         12
3         10
4          8
          ..
533125     3
533126     4
533127     5
533128     6
533129     8
Name: PlaceInt, Length: 533130, dtype: int64

In [11]:
import numpy as np 

def stone_and_pounds_to_kg(weight: str):

    stone_to_kg = 6.35029
    pound_to_kg = 0.453592

    weight_split = weight.split('-')

    try:
        stones = float(weight_split[0])
        pounds = float(weight_split[1])

        return (stone_to_kg * stones) + (pound_to_kg * pounds)
    except:
        return np.NaN
    
def fractional_odds_to_implied_probability(odds: str):

    if type(odds) == float:
        return np.nan
    
    odds = odds.replace('F', '')
    odds = odds.replace('C', '')
    odds = odds.replace('J', '')
    odds = odds.replace('Evens', '1/1')
    odds = odds.replace('Evs', '1/1')
    odds_split = odds.split('/')

    try:
        return float(odds_split[1]) / (float(odds_split[0]) + float(odds_split[1]))
    except: 
        print(odds)

def decimal_odds_to_implied_probability(odds: float):
    return 1 / odds

def clean_non_historic_attributes(df: pd.DataFrame):

    pace_mean = df['Pace'].mean()
    df['Pace'] = df['Pace']

    df['WeightKg'] = df['Weight'].apply(lambda x: stone_and_pounds_to_kg(x))
    weight_mean = df['WeightKg'].mean()
    df['WeightKg'] = df['WeightKg']

    offR_max = df['OffR'].max()
    df['OffR'] = df['OffR']

    df['TimeSinceLastRaceForHorse'] = (
        df.groupby("Horse")["DateTime"]
        .transform(lambda x: x.diff().apply(lambda x: x.total_seconds() / 60))
    )

    time_since_last_race_max = df['TimeSinceLastRaceForHorse'].max()

    df['TimeSinceLastRaceForHorse'] = df['TimeSinceLastRaceForHorse']

    df['TimeSinceLastRaceForJockey'] = (
        df.groupby("Jockey")["DateTime"]
        .transform(lambda x: x.diff().apply(lambda x: x.total_seconds() / 60))
    )

    time_since_last_race_max = df['TimeSinceLastRaceForJockey'].max()

    df['TimeSinceLastRaceForJockey'] = df['TimeSinceLastRaceForJockey']

    df['industry_implied_prob'] = df['Ind SP'].apply(lambda x: fractional_odds_to_implied_probability(x))
    df['betfair_implied_prob'] = df['BSP'].apply(lambda x: 1/x if x > 0 else x)

    df['betfair_implied_prob'] = df[['industry_implied_prob', 'betfair_implied_prob']].apply(lambda x: x['betfair_implied_prob'] if x['betfair_implied_prob'] > 0 else x['industry_implied_prob'], axis=1)
    df['industry_implied_prob'] = df[['industry_implied_prob', 'betfair_implied_prob']].apply(lambda x: x['industry_implied_prob'] if x['industry_implied_prob'] > 0 else x['betfair_implied_prob'], axis=1)

    df['betfair_implied_prob_norm'] = df.groupby(['DateTime', 'Track Name'])["betfair_implied_prob"].transform(lambda x: x / x.sum())
    df['industry_implied_prob_norm'] = df.groupby(['DateTime', 'Track Name'])["industry_implied_prob"].transform(lambda x: x / x.sum())

    df['betfair_implied_prob_norm_inv'] = 1 / df['betfair_implied_prob_norm']
    df['industry_implied_prob_norm_inv'] = 1 / df['industry_implied_prob_norm']

    return df

In [12]:
df = get_historical_for_group(df, by="Horse")
df = get_historical_for_group(df, by="Jockey")
df = get_historical_for_group(df, by="Trainer")
df = get_historical_for_group(df, by=["Horse", "Jockey"])
df = get_historical_for_group(df, by=["Horse", "Trainer"])
df = get_historical_for_group(df, by=["Jockey", "Trainer"])
df = get_historical_for_group(df, by=["Horse", "Going"])
df = get_historical_for_group(df, by=["Jockey", "Going"])
df = get_historical_for_group(df, by=["Horse", "Distance"])
df = get_historical_for_group(df, by=["Jockey", "Distance"])
df = get_historical_for_group(df, by=["Horse", "distance_bin"])
df = get_historical_for_group(df, by=["Jockey", "distance_bin"])

/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/2757279380.py:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda x: x.shift(1).cumsum()).fillna(0)
/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/2757279380.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda x: x.shift(1).cumsum()).fillna(0)
/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/2757279380.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a futur

In [18]:
df = clean_non_historic_attributes(df)

/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/1037566358.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['WeightKg'] = df['Weight'].apply(lambda x: stone_and_pounds_to_kg(x))
/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/1037566358.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['TimeSinceLastRaceForHorse'] = (
/var/folders/dy/mtplz5t11tv7w14rf5z7nx_r0000gn/T/ipykernel_28931/1037566358.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [ ]:
df['15 mins'] = df[['15 mins', '15 Mins']].apply(lambda x: x['15 mins'] if x['15 mins'] > 0 else x['15 Mins'], axis=1)
df['1 min'] = df['1 min ']

In [50]:
column_names_data.add('betfair_implied_prob_norm')
column_names_data.add('betfair_implied_prob_norm_inv')
column_names_data.add('industry_implied_prob_norm')
column_names_data.add('industry_implied_prob_norm_inv')

In [22]:
df = df.iloc[indx_with_better_data:]
columns_x = list(column_names_data)
columns_y = ['Horse', 'win', 'DateTime', 'Track Name', 'PlaceInt', 'PlaceNorm', 'First Trading Price', '15 mins', '10 mins', '5 mins', '3 mins', '2 mins', '1 min ', 'Post Time']
df[columns_x + columns_y].to_csv('ProcessedDatasetNulls.csv')

In [21]:
df['Horse']

23550     Keepthebestsideout
23551                 Pillar
23552             Red Cymbal
23553         Maggie Thunder
23554           Is That Love
                 ...        
533125        Onslow Gardens
533126        Sense Of Worth
533127          Timetobenice
533128              Kracking
533129        African Spirit
Name: Horse, Length: 509580, dtype: object

In [1]:
df['PlaceInt']

NameError: name 'df' is not defined